# Start

NEW TASKS:
* [X] Seq2Seq: sort by src_len and unsort output --> ensure output matches with trg
* [X] Pivot model: ensure it works for $n$ seq2seq models
* [ ] Triang model: ensure outputs from all submodels match

In [ ]:
# piv_endefr_74kset_2.pt using PivotModel in bentrevett/pytorch-seq2seq-OLD.ipynb

In [ ]:
# https://github.com/bentrevett/pytorch-seq2seq/blob/master/4%20-%20Packed%20Padded%20Sequences%2C%20Masking%2C%20Inference%20and%20BLEU.ipynb
# based on https://gmihaila.github.io/tutorial_notebooks/pytorchtext_bucketiterator/#dataset-class

In [ ]:
piv_langs = ['it']
langs = ['en', 'fr'] + piv_langs
DIR_PATH = '/content/gdrive/MyDrive/Colab Notebooks/eaai24'

# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip3 install torch==1.8.2 torchvision==0.9.2 torchaudio==0.8.2 --extra-index-url https://download.pytorch.org/whl/lts/1.8/cu111 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 991.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.14.1 requires torch==1.13.1, but you have torch 1.8.2+cu111 which is incompatible.


In [ ]:
import torch
print(torch.__version__)

1.8.2+cu111


In [ ]:
!pip install torchtext==0.9 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 MB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.9.2+cu111 requires torch==1.8.2, but you have torch 1.8.0 which is incompatible.
torchaudio 0.8.2 requires torch==1.8.2, but you have torch 1.8.0 which is incompatible.


In [ ]:
if 'en' in langs:
  !python -m spacy download en_core_web_sm -q
if 'de' in langs:
  !python -m spacy download de_core_news_sm -q
if 'fr' in langs:
  !python -m spacy download fr_core_news_sm -q
if 'it' in langs:
  !python -m spacy download it_core_news_sm -q
if 'es' in langs:
  !python -m spacy download es_core_news_sm -q
if 'pt' in langs:
  !python -m spacy download pt_core_news_sm -q
if 'ro' in langs:
  !python -m spacy download ro_core_news_sm -q

2023-03-17 15:04:46.209344: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 15:04:46.363442: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-17 15:04:47.113026: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-17 15:04:47.113129: W tensorflow/compiler/xla/stream_executor

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.data import Field, BucketIterator
from torchtext.legacy.data import Dataset, Example
from torchtext.data.metrics import bleu_score

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time
import pickle
from tqdm import tqdm

# My Section

## Setup

In [ ]:
FIELD_DICT = {}
if 'en' in langs:
  spacy_en = spacy.load('en_core_web_sm')
  def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]
  EN_FIELD = Field(tokenize = tokenize_en, init_token = '<sos>', eos_token = '<eos>', lower = True, include_lengths = True)
  FIELD_DICT['en'] = EN_FIELD
if 'de' in langs:
  spacy_de = spacy.load('de_core_news_sm')
  def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]
  DE_FIELD = Field(tokenize = tokenize_de, init_token = '<sos>', eos_token = '<eos>', lower = True, include_lengths = True)
  FIELD_DICT['de'] = DE_FIELD
if 'fr' in langs:
  spacy_fr = spacy.load('fr_core_news_sm')
  def tokenize_fr(text):
    return [tok.text for tok in spacy_fr.tokenizer(text)]
  FR_FIELD = Field(tokenize = tokenize_fr, init_token = '<sos>', eos_token = '<eos>', lower = True, include_lengths = True)
  FIELD_DICT['fr'] = FR_FIELD
if 'it' in langs:
  spacy_it = spacy.load('it_core_news_sm')
  def tokenize_it(text):
    return [tok.text for tok in spacy_it.tokenizer(text)]
  IT_FIELD = Field(tokenize = tokenize_it, init_token = '<sos>', eos_token = '<eos>', lower = True, include_lengths = True)
  FIELD_DICT['it'] = IT_FIELD
if 'es' in langs:
  spacy_es = spacy.load('es_core_news_sm')
  def tokenize_es(text):
    return [tok.text for tok in spacy_es.tokenizer(text)]
  ES_FIELD = Field(tokenize = tokenize_es, init_token = '<sos>', eos_token = '<eos>', lower = True, include_lengths = True)
  FIELD_DICT['es'] = ES_FIELD
if 'pt' in langs:
  spacy_pt = spacy.load('pt_core_news_sm')
  def tokenize_pt(text):
    return [tok.text for tok in spacy_pt.tokenizer(text)]
  PT_FIELD = Field(tokenize = tokenize_pt, init_token = '<sos>', eos_token = '<eos>', lower = True, include_lengths = True)
  FIELD_DICT['pt'] = PT_FIELD
if 'ro' in langs:
  spacy_ro = spacy.load('ro_core_news_sm')
  def tokenize_ro(text):
    return [tok.text for tok in spacy_ro.tokenizer(text)]
  RO_FIELD = Field(tokenize = tokenize_ro, init_token = '<sos>', eos_token = '<eos>', lower = True, include_lengths = True)
  FIELD_DICT['ro'] = RO_FIELD

## Data

In [ ]:
BATCH_SIZE = 16

train_len = 64000
valid_len = 3200
test_len = 6400

train_pt = train_len
valid_pt = train_pt + valid_len
test_pt = valid_pt + test_len

print(train_pt, valid_pt, test_pt)

64000 67200 73600


In [ ]:
# https://discuss.pytorch.org/t/how-to-save-and-load-torchtext-data-field-build-vocab-result/50407/3
def save_vocab(vocab, path):
  with open(path, 'w+', encoding='utf-8') as f:
    for token, index in vocab.stoi.items():
      f.write(f'{index}\t{token}\n')
def read_vocab(path):
  vocab = dict()
  with open(path, 'r', encoding='utf-8') as f:
    for line in f:
      index, token = line.split('\t')
      vocab[token] = int(index)
  return vocab

In [ ]:
# with open('/content/gdrive/MyDrive/Colab Notebooks/eaai24/Datasets/endefr_75kpairs_2k5-freq-words.pkl', 'rb') as f:
# with open('/content/gdrive/MyDrive/Colab Notebooks/eaai24/Datasets/enfr_160kpairs_2k5-freq-words.pkl', 'rb') as f:
#   data = pickle.load(f)
# data[80], len(data)

In [ ]:
# EnDeFrItEsPtRo-60k-most10k-1.pkl is most suitable. Each lang has ~5-7k words
# EnDeFrItEsPtRo-76k-most5k.pkl: each lang has 6->12k words (too much. use this in the future)

In [ ]:
dataname = 'EnDeFrItEsPtRo-76k-most5k.pkl'
with open(f'{DIR_PATH}/Datasets/{dataname}', 'rb') as f:
  data = pickle.load(f)
data[8], len(data)

({'en': 'What is the result?',
  'de': 'Und wie sehen die Ergebnisse aus?',
  'fr': 'Quelle en est la conséquence ?',
  'it': 'Quali sono i risultati?',
  'es': '¿Cuáles son los resultados?',
  'pt': 'Quais são os resultados?',
  'ro': 'Care este rezultatul?'},
 76245)

In [ ]:
# data_set = [[pair['en'], pair['it'], pair['fr']] for pair in data]
# FIELDS = [('en', EN_FIELD), ('it', IT_FIELD), ('fr', FR_FIELD)]
data_set = [[pair[lang] for lang in langs] for pair in data]
FIELDS = [(lang, FIELD_DICT[lang]) for lang in langs]
train_examples = list(map(lambda x: Example.fromlist(list(x), fields=FIELDS), data_set[: train_pt]))
valid_examples = list(map(lambda x: Example.fromlist(list(x), fields=FIELDS), data_set[train_pt : valid_pt]))
test_examples = list(map(lambda x: Example.fromlist(list(x), fields=FIELDS), data_set[valid_pt : test_pt]))

In [ ]:
train_dt = Dataset(train_examples, fields=FIELDS)
valid_dt = Dataset(valid_examples, fields=FIELDS)
test_dt = Dataset(test_examples, fields=FIELDS)

In [ ]:
# EN_FIELD.build_vocab(train_dt, min_freq = 2)
# IT_FIELD.build_vocab(train_dt, min_freq = 2)
# FR_FIELD.build_vocab(train_dt, min_freq = 2)
# len(EN_FIELD.vocab), len(IT_FIELD.vocab), len(FR_FIELD.vocab)
for lang in langs:
  FIELD_DICT[lang].build_vocab(train_dt, min_freq = 2)
  print(f'{lang}: {len(FIELD_DICT[lang].vocab)}')

en: 6964
fr: 9703
it: 10712


In [ ]:
# save_vocab(EN_FIELD.vocab, f'{DIR_PATH}/Datasets/EnDeFrItEsPtRo-76k-most5k-en_vocab.txt')
# EN_FIELD.vocab = read_vocab(f'{DIR_PATH}/Datasets/EnDeFrItEsPtRo-76k-most5k-en_vocab.txt')
# FR_FIELD.vocab = read_vocab(f'{DIR_PATH}/Datasets/EnDeFrItEsPtRo-76k-most5k-fr_vocab.txt')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dt, valid_dt, test_dt),
     batch_size = BATCH_SIZE,
     sort_within_batch = True,
     sort_key = lambda x : len(x.en),
     device = device)

In [ ]:
# for i, batch in enumerate(train_iterator):
#   break
# print(batch.en[0].shape, batch.en[1])
# print(batch.fr[0].shape, batch.fr[1])
# print(batch.fields)

In [ ]:
# src_sent, piv_sent, trg_sent = [], [], []
# for i in batch.en[0][: , 0]:
#   src_sent.append(EN_FIELD.vocab.itos[i])
# for i in batch.fr[0][:, 0]:
#   trg_sent.append(FR_FIELD.vocab.itos[i])
# print(' '.join(src_sent))
# print(' '.join(trg_sent))

In [ ]:
# for i in range(10):
#   print(EN_FIELD.vocab.itos[i], FR_FIELD.vocab.itos[i])

## Model

The correct implementation of that [paper](https://arxiv.org/pdf/1409.0473.pdf) should be like this: [repo](https://github.com/graykode/nlp-tutorial#4-attention-mechanism) - [colab](https://colab.research.google.com/github/graykode/nlp-tutorial/blob/master/4-2.Seq2Seq(Attention)/Seq2Seq(Attention).ipynb). My method is a bit different, actually it is implemented based on [Luong et. al.](https://arxiv.org/pdf/1508.04025.pdf) with Global Attention using concat method ([implement in tf](https://github.com/philipperemy/keras-attention/blob/master/attention/attention.py)).

### Encoder

In [ ]:
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
    super().__init__()
    self.embedding = nn.Embedding(input_dim, emb_dim)
    self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
    self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, src, src_len):
    #src = [src len, batch size]
    #src_len = [batch size]
    embedded = self.dropout(self.embedding(src))  #embedded = [src len, batch size, emb dim]

    #need to explicitly put lengths on cpu!
    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len.to('cpu'))

    #  when the input is a pad token are all zeros
    packed_outputs, hidden = self.rnn(packed_embedded)
    #packed_outputs is a packed sequence containing all hidden states
    #hidden is now from the final non-padded element in the batch

    outputs, len_list = nn.utils.rnn.pad_packed_sequence(packed_outputs) #outputs is now a non-packed sequence, all hidden states obtained
    #  when the input is a pad token are all zeros

    #outputs = [src len, batch size, hid dim * num directions]
    #hidden = [n layers * num directions, batch size, hid dim]

    #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
    #outputs are always from the last layer

    #hidden [-2, :, : ] is the last of the forwards RNN
    #hidden [-1, :, : ] is the last of the backwards RNN

    #initial decoder hidden is final hidden state of the forwards and backwards
    #  encoder RNNs fed through a linear layer
    hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

    #outputs = [src len, batch size, enc hid dim * 2]
    #hidden = [batch size, dec hid dim]
    return outputs, hidden

### Attn

In [ ]:
class Attention(nn.Module):
  def __init__(self, enc_hid_dim, dec_hid_dim):
    super().__init__()
    self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
    self.v = nn.Linear(dec_hid_dim, 1, bias = False)

  def forward(self, hidden, encoder_outputs, mask):
    #hidden = [batch size, dec hid dim]
    #encoder_outputs = [src len, batch size, enc hid dim * 2]
    batch_size = encoder_outputs.shape[1]
    src_len = encoder_outputs.shape[0]

    #repeat decoder hidden state src_len times
    hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)  #hidden = [batch size, src len, dec hid dim]
    encoder_outputs = encoder_outputs.permute(1, 0, 2)  #encoder_outputs = [batch size, src len, enc hid dim * 2]
    energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) #energy = [batch size, src len, dec hid dim]
    # eij = a(si−1, hj )

    attention = self.v(energy).squeeze(2) #attention = [batch size, src len]
    attention = attention.masked_fill(mask == 0, -1e10)
    return F.softmax(attention, dim = 1)

### Decoder

In [ ]:
class Decoder(nn.Module):
  def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
    super().__init__()
    self.output_dim = output_dim
    self.attention = attention
    self.embedding = nn.Embedding(output_dim, emb_dim)
    self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
    self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, input, hidden, encoder_outputs, mask):
    #input = [batch size]
    #hidden = [batch size, dec hid dim]
    #encoder_outputs = [src len, batch size, enc hid dim * 2]
    #mask = [batch size, src len]
    input = input.unsqueeze(0)  #input = [1, batch size]
    embedded = self.dropout(self.embedding(input))  #embedded = [1, batch size, emb dim]

    a = self.attention(hidden, encoder_outputs, mask) #a = [batch size, src len]
    a = a.unsqueeze(1)  #a = [batch size, 1, src len]

    encoder_outputs = encoder_outputs.permute(1, 0, 2)  #encoder_outputs = [batch size, src len, enc hid dim * 2]

    weighted = torch.bmm(a, encoder_outputs)  #weighted = [batch size, 1, enc hid dim * 2]
    weighted = weighted.permute(1, 0, 2)  #weighted = [1, batch size, enc hid dim * 2]

    rnn_input = torch.cat((embedded, weighted), dim = 2)  #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]

    output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
    #output = [seq len, batch size, dec hid dim * n directions]
    #hidden = [n layers * n directions, batch size, dec hid dim]

    #seq len, n layers and n directions will always be 1 in this decoder, therefore:
    #output = [1, batch size, dec hid dim]
    #hidden = [1, batch size, dec hid dim]
    #this also means that output == hidden
    assert (output == hidden).all()

    embedded = embedded.squeeze(0)
    output = output.squeeze(0)
    weighted = weighted.squeeze(0)

    prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))  #prediction = [batch size, output dim]
    return prediction, hidden.squeeze(0), a.squeeze(1)

### Seq2Seq

In [ ]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, src_pad_idx, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_pad_idx = src_pad_idx
    self.device = device

  def create_mask(self, src):
    mask = (src != self.src_pad_idx).permute(1, 0)
    return mask

  def forward(self, datas, criterion=None, teacher_forcing_ratio = 0.5):
    #src = [src len, batch size]
    #src_len = [batch size]
    #trg = [trg len, batch size]
    #trg_len = [batch size]
    #teacher_forcing_ratio is probability of using trg to be input else prev output to be input for next prediction.
    (src, src_len), (trg, _) = datas
    batch_size = src.shape[1]
    trg_len = trg.shape[0]
    trg_vocab_size = self.decoder.output_dim

    # SORT
    sort_ids, unsort_ids = self.sort_by_sent_len(src_len)
    src, src_len, trg = src[:, sort_ids], src_len[sort_ids], trg[:, sort_ids]

    #tensor to store decoder outputs
    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

    #encoder_outputs is all hidden states of the input sequence, back and forwards
    #hidden is the final forward and backward hidden states, passed through a linear layer
    encoder_outputs, hidden = self.encoder(src, src_len)

    #first input to the decoder is the <sos> tokens
    input = trg[0,:]

    mask = self.create_mask(src)  #mask = [batch size, src len]

    for t in range(1, trg_len):
      #insert input token embedding, previous hidden state, all encoder hidden states and mask
      #receive output tensor (predictions) and new hidden state
      output, hidden, _ = self.decoder(input, hidden, encoder_outputs, mask)

      #place predictions in a tensor holding predictions for each token
      outputs[t] = output

      #if teacher forcing, use actual next token as next input. Else, use predicted token
      input = trg[t] if random.random() < teacher_forcing_ratio else output.argmax(1)

    if criterion != None:
      loss = self.compute_loss(outputs, trg, criterion)
      return loss, outputs[:, unsort_ids, :]
    return outputs[:, unsort_ids, :]

  def compute_loss(self, output, trg, criterion):
    #output = (trg_len, batch_size, trg_vocab_size)
    #trg = [trg len, batch size]
    output = output[1:].view(-1, output.shape[-1])  #output = [(trg len - 1) * batch size, output dim]
    trg = trg[1:].view(-1)  #trg = [(trg len - 1) * batch size]
    loss = criterion(output, trg)
    return loss

  # NEWLY ADDED ##########################
  def sort_by_sent_len(self, sent_len):
    _, sort_ids = sent_len.sort(descending=True)
    unsort_ids = sort_ids.argsort()
    return sort_ids, unsort_ids
  # END ADDED ############################

### Pivot model (update)

In [ ]:
class PivotSeq2Seq(nn.Module):
  def __init__(self, models: list, fields: list, device, alpha=1.1, lamda=0.75):
    super().__init__()
    self.num_model = len(models)
    self.fields = fields
    self.num_field = len(fields)
    self.device = device
    self.alpha = alpha
    self.lamda = lamda

    for i in range(self.num_model):
      self.add_module(f'model_{i}', models[i])

    assert len(models)+1 == len(fields), f"Not enough Fields for models: num_field={len(fields)} != {len(models)+1}"

  def forward(self, datas: list, criterions=None, teacher_forcing_ratio=0.5):
    '''
    datas: list of data: [(src, src_len), (piv1, piv_len1), ... , (pivM, piv_lenM), (trg, trg_len)] given M models
      src = [src len, batch_size]
      src_len = [batch_size]
      ...
      trg = [trg len, batch_size]
      trg_len = [batch_size]
    criterions: list of criterion for each model
    '''
    if criterions != None:
      loss_list, output_list = self.run(datas, criterions, teacher_forcing_ratio)
      total_loss = self.compute_loss(loss_list)
      return total_loss, output_list[-1]
    else:
      criterions = [None for _ in range(self.num_model)]
      _, output_list = self.run(datas, criterions, teacher_forcing_ratio)
      return output_list[-1]

  def run(self, datas, criterions, teacher_forcing_ratio):
    assert self.num_model+1 == len(datas), f"Not enough datas for models: data_len={len(datas)} != {self.num_model+1}"
    assert self.num_model == len(criterions), f'Criterions must have for each model: num_criterion={len(criterions)} != {self.num_model}'

    output_list, loss_list = [], []
    for i in range(self.num_model):
      isForceOn = True if i==0 else random.random() < teacher_forcing_ratio # 1st model must always use src

      # GET NEW INPUT
      src, src_len = datas[i] if isForceOn else self.process_output(output_list[-1], self.fields[i+1])
      trg, trg_len = datas[i+1]

      # FORWARD MODEL
      model = getattr(self, f'model_{i}') # Seq2Seq model already sort src by src_len in forward
      data = [(src, src_len), (trg, trg_len)]
      criterion = criterions[i]
      output = model(data, criterion, 0 if criterion==None else teacher_forcing_ratio)

      if criterion == None:
        output_list.append(output)
      else:
        assert len(output) == 2, 'With criterion, model should return loss & prediction'
        loss, out = output
        loss_list.append(loss)
        output_list.append(out)

    return loss_list, output_list

  def compute_loss(self, loss_list):
    total_loss = 0.0
    for i in range(len(loss_list) - 1): # except final output
      total_loss += loss_list[i]
    total_loss += self.alpha*loss_list[-1]
    return total_loss + self.lamda*self.compute_embed_loss()

  def compute_embed_loss(self):
    embed_loss = 0.0
    for i in range(1, self.num_model):
      model1 = getattr(self, f'model_{i-1}')
      model2 = getattr(self, f'model_{i}')
      embed_loss += torch.sum(F.pairwise_distance(model1.decoder.embedding.weight, model2.encoder.embedding.weight, p=2))
    return embed_loss

  def sort_by_src_len(self, piv, piv_len, datas): # piv = [piv_len, batch_size]
    piv_len, sorted_ids = piv_len.sort(descending=True)
    sorted_datas = [(sent[:, sorted_ids], sent_len[sorted_ids]) for (sent, sent_len) in datas]
    return piv[:, sorted_ids], piv_len, sorted_datas  # piv sorted along batch_size

  def process_output(self, output, piv_field):
    # output = [trg len, batch size, output dim]
    # trg = [trg len, batch size]
    # Process output1 to be input for model2
    seq_len, N, _ = output.shape
    tmp_out = output.argmax(2)  # tmp_out = [seq_len, batch_size]
    # re-create pivot as src for model2
    piv = torch.zeros_like(tmp_out).type(torch.long).to(output.device)
    piv[0, :] = torch.full_like(piv[0, :], piv_field.vocab.stoi[piv_field.init_token])  # fill all first idx with sos_token

    for i in range(1, seq_len):  # for each i in seq_len
      # if tmp_out's prev is eos_token, replace w/ pad_token, else current value
      eos_mask = (tmp_out[i-1, :] == piv_field.vocab.stoi[piv_field.eos_token])
      piv[i, :] = torch.where(eos_mask, piv_field.vocab.stoi[piv_field.pad_token], tmp_out[i, :])
      # if piv's prev is pad_token, replace w/ pad_token, else current value
      pad_mask = (piv[i-1, :] == piv_field.vocab.stoi[piv_field.pad_token])
      piv[i, :] = torch.where(pad_mask, piv_field.vocab.stoi[piv_field.pad_token], piv[i, :])

    # Trim down extra pad tokens
    tensor_list = [piv[i] for i in range(seq_len) if not all(piv[i] == piv_field.vocab.stoi[piv_field.pad_token])]  # tensor_list = [new_seq_len, batch_size]
    piv = torch.stack([x for x in tensor_list], dim=0).type(torch.long).to(output.device)
    assert not all(piv[-1] == piv_field.vocab.stoi[piv_field.pad_token]), 'Not completely trim down tensor'

    # get seq_id + eos_tok id of each sequence
    piv_ids, eos_ids = (piv.permute(1, 0) == piv_field.vocab.stoi[piv_field.eos_token]).nonzero(as_tuple=True)  # piv_len = [N]
    piv_len = torch.full_like(piv[0], seq_len).type(torch.long)  # init w/ longest seq
    piv_len[piv_ids] = eos_ids + 1 # seq_len = eos_tok + 1

    return piv, piv_len

### Triangulate model

In [ ]:
class TriangSeq2Seq(nn.Module):
  def __init__(self, models: list, output_dim, device, alpha=1.1, method='weighted', train_backbone=True):
    # output_dim = trg vocab size
    super().__init__()
    self.num_model = len(models)
    self.output_dim = output_dim
    self.device = device
    self.alpha = alpha
    self.method = method
    self.train_backbone = train_backbone
    if method=='weighted':
      self.head = nn.Sequential(
          nn.ReLU(),
          nn.Linear(output_dim*self.num_model, output_dim)
      )
    elif method=='weighted_1':
      self.head = nn.Sequential(
          nn.Linear(self.num_model, self.num_model*2),
          nn.ReLU(),
          nn.Dropout(0.2),
          nn.Linear(self.num_model*2, self.num_model*2),
          nn.ReLU(),
          nn.Dropout(0.2),
          nn.Linear(self.num_model*2, 1),
      )
    elif method=='weighted_2':
      self.head = nn.Sequential(
          nn.Linear(self.num_model, self.num_model),
          nn.ReLU(),
          nn.Linear(self.num_model, self.num_model),
          nn.ReLU(),
          nn.Linear(self.num_model, self.num_model),
      )
    for i in range(self.num_model):
      for param in models[i].parameters():
        param.requires_grad = train_backbone
      self.add_module(f'model_{i}', models[i])

  def forward(self, datas: dict, criterions=None, teacher_forcing_ratio=0.5):
    '''
    datas: dict of data:
      {"model_0": (src, src_len, trg, trg_len), "model_1": [(src, src_len), (piv, piv_len), (trg, trg_len)], ..., "TRG": (trg, trg_len)}
      src = [src len, batch size]
      src_len = [batch size]
    criterions: dict of criterions
      {"model_0": criterion_0, "model_1": criterion_1, ..., "TRG": criterion_M}
    '''
    if criterions != None:
      loss_list, output_list = self.run(datas, criterions, teacher_forcing_ratio)
      final_out = self.get_final_pred(output_list)
      total_loss = self.alpha*self.compute_final_pred_loss(final_out, datas["TRG"], criterions["TRG"]) + self.compute_submodels_loss(loss_list)
      return total_loss, final_out
    else:
      criterions = {f'model_{i}':None for i in range(self.num_model)}
      criterions['TRG'] = None
      loss_list, output_list = self.run(datas, criterions, teacher_forcing_ratio)
      final_out = self.get_final_pred(output_list)
      return final_out

  def run(self, datas, criterions, teacher_forcing_ratio):
    assert self.num_model+1 == len(datas), f"Not enough datas for models: data_len={len(datas)} != {self.num_model+1}"  # include 'TRG'
    assert self.num_model+1 == len(criterions), f'Criterions must have for each model: num_criterion={len(criterions)} != {self.num_model+1}' # include 'TRG'

    output_list = []
    loss_list = []
    for i in range(self.num_model):
      data = datas[f'model_{i}']
      model = getattr(self, f'model_{i}')
      criterion = criterions[f'model_{i}']
      output = model(data, criterion, 0 if criterion==None else teacher_forcing_ratio)

      if criterion == None:
        output_list.append(output)
      else:
        assert len(output) == 2, 'With criterion, model should return loss & prediction'
        loss_list.append(output[0])
        output_list.append(output[1])

    return loss_list, output_list

  def compute_submodels_loss(self, loss_list):
    total_loss = 0.0
    for loss in loss_list:
      total_loss += loss
    return total_loss

  def compute_final_pred_loss(self, output, data, criterion):
    #output = (trg_len, batch_size, trg_vocab_size)
    #data = [trg, trg_len]  # trg.shape = [seq_len, batch_size]
    trg, _ = data
    output = output[1:].reshape(-1, output.shape[-1])  #output = [(trg len - 1) * batch size, output dim]
    trg = trg[1:].reshape(-1)  #trg = [(trg len - 1) * batch size]
    loss = criterion(output, trg)
    return loss

  def get_final_pred(self, output_list):  # output_list[0] shape = [seq_len, N, out_dim]
    # assert all([output_list[i].shape == output_list[i-1].shape for i in range(1, len(output_list))]), 'all outputs must match shape [seq_len, N, out_dim]'
    seq_len, N, out_dim = output_list[0].shape
    if self.method=='weighted':
      linear_in = torch.cat([out for out in output_list], dim=-1) # linear_in = [seq_len, N, out_dim * num_model]. Note that num_model = len(output_list)
      final_out = self.head(linear_in)  # final_out = [seq_len, N, out_dim]
      return final_out
    elif self.method=='weighted_1':
      output_list = [out.permute(1, 0, 2).reshape(N, -1) for out in output_list]  # [N, seq_len, out_dim] --> [N, seq_len*out_dim]
      final_out = self.head(torch.stack(output_list, dim=-1)).squeeze(-1)  # [N, seq_len*out_dim, num_model] --> [N, seq_len*out_dim, 1] --> [N, seq_len*out_dim]
      return final_out.reshape(N, seq_len, out_dim).permute(1, 0, 2) # [N, seq_len, out_dim] --> [seq_len, N, out_dim]
    elif self.method=='weighted_2':
      output_list = [out.permute(1, 0, 2).argmax(-1) for out in output_list]  # [N, seq_len, out_dim] --> [N, seq_len]
      inp = torch.stack(output_list, dim=-1).type(torch.float32)  # [N, seq_len, num_model]
      out = self.head(inp)
      finalout = torch.gather(inp, -1, out.argmax(-1, True)).squeeze(-1).type(torch.long)
      finalout = F.one_hot(finalout, num_classes=self.output_dim)
      return finalout.permute(1, 0, 2).type(torch.float32)
    elif self.method=='max': # get max along seq_len b/w all outputs
      output_list = [out.permute(1, 0, 2).reshape(N, -1) for out in output_list]  # [N, seq_len, out_dim] --> [N, seq_len*out_dim]
      final_out, _ = torch.max(torch.stack(output_list, dim=-1), dim=-1)  # [N, seq_len*out_dim, num_model] --> [N, seq_len*out_dim]
      return final_out.reshape(N, seq_len, out_dim).permute(1, 0, 2) # [N, seq_len, out_dim] --> [seq_len, N, out_dim]
    else:
      return output_list[0]

## Train func

In [ ]:
def update_trainlog(data: list, filename: str=f'{DIR_PATH}/training_log.txt'):
  ''' Update training log w/ new losses
  Args:
      data (List): a list of infor for many epochs as tuple, each tuple has model_name, loss, etc.
      filename (String): path + file_name
  Return:
      None: new data is appended into train-log
  '''
  with open(filename, 'a') as f: # save
    for epoch in data:
      f.write(','.join(epoch))
      f.write("\n")
  print('update_trainlog SUCCESS')
  return []
def init_weights(m):
  for name, param in m.named_parameters():
    if 'weight' in name:
      nn.init.normal_(param.data, mean=0, std=0.01)
    else:
      nn.init.constant_(param.data, 0)
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

### Seq2Seq

In [ ]:
def trainSeq2Seq(model, iterator, optimizer, criterion, clip):
  model.train()
  epoch_loss = 0.0
  for batch in tqdm(iterator):
    optimizer.zero_grad()
    datas = [batch.en, batch.fr]
    loss, _ = model(datas, criterion, 0.5)

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    epoch_loss += loss.item()
  return epoch_loss / len(iterator)

def evaluateSeq2Seq(model, iterator, criterion):
  model.eval()
  epoch_loss = 0.0
  with torch.no_grad():
    for batch in tqdm(iterator):
      datas = [batch.en, batch.fr]
      loss, _ = model(datas, criterion, 0) # turn off teacher forcing
      epoch_loss += loss.item()
    return epoch_loss / len(iterator)

### Pivot

In [ ]:
def trainPivot(model, iterator, optimizer, criterions, clip):
  model.train()
  epoch_loss = 0.0
  for batch in tqdm(iterator):
    optimizer.zero_grad()
    model_inputs = [batch.en, vars(batch)['it'], batch.fr]
    loss, _ = model(model_inputs, criterions, 0.5)

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    epoch_loss += loss.item()
  return epoch_loss / len(iterator)

def evaluatePivot(model, iterator, criterions):
  model.eval()
  epoch_loss = 0.0
  with torch.no_grad():
    for batch in tqdm(iterator):
      model_inputs = [batch.en, vars(batch)['it'], batch.fr]
      loss, _ = model(model_inputs, criterions, 0)
      epoch_loss += loss.item()
    return epoch_loss / len(iterator)

### Triangulate

In [ ]:
# for triangulate model
def trainTriang(model, iterator, optimizer, criterions, clip):
  model.train()
  epoch_loss = 0.0
  for batch in tqdm(iterator):
    optimizer.zero_grad()
    model_inputs = {
        'model_0': [batch.en, batch.fr],
        'model_1': [batch.en, vars(batch)['it'], batch.fr],
        'TRG': batch.fr
    }
    loss, _ = model(model_inputs, criterions, 0.5)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    epoch_loss += loss.item()
  return epoch_loss / len(iterator)

def evaluateTriang(model, iterator, criterions):
  model.eval()
  epoch_loss = 0.0
  with torch.no_grad():
    for batch in tqdm(iterator):
      model_inputs = {
        'model_0': [batch.en, batch.fr],
        'model_1': [batch.en, vars(batch)['it'], batch.fr],
        'TRG': batch.fr
      }
      loss, _ = model(model_inputs, criterions, 0)
      epoch_loss += loss.item()
    return epoch_loss / len(iterator)

## Train

### Train model

#### Seq2Seq

In [ ]:
# For 2 langs
EMB_DIM = 256
HID_DIM = 512
DROPOUT = 0.5

INPUT_DIM = len(EN_FIELD.vocab)
OUTPUT_DIM = len(FR_FIELD.vocab)
SRC_PAD_IDX = EN_FIELD.vocab.stoi[EN_FIELD.pad_token]

attn = Attention(HID_DIM, HID_DIM)
enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM, HID_DIM, DROPOUT)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, HID_DIM, DROPOUT, attn)

model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

#### Pivot

In [ ]:
EMB_DIM = 256
HID_DIM = 512
DROPOUT = 0.5

INPUT_DIM = len(EN_FIELD.vocab)
PIV_DIM = len(ES_FIELD.vocab)
OUTPUT_DIM = len(FR_FIELD.vocab)

SRC_PAD_IDX = EN_FIELD.vocab.stoi[EN_FIELD.pad_token]
attn1 = Attention(HID_DIM, HID_DIM)
enc1 = Encoder(INPUT_DIM, EMB_DIM, HID_DIM, HID_DIM, DROPOUT)
dec1 = Decoder(PIV_DIM, EMB_DIM, HID_DIM, HID_DIM, DROPOUT, attn1)
model1 = Seq2Seq(enc1, dec1, SRC_PAD_IDX, device).to(device)

PIV_PAD_IDX = ES_FIELD.vocab.stoi[ES_FIELD.pad_token]
attn2 = Attention(HID_DIM, HID_DIM)
enc2 = Encoder(PIV_DIM, EMB_DIM, HID_DIM, HID_DIM, DROPOUT)
dec2 = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, HID_DIM, DROPOUT, attn2)
model2 = Seq2Seq(enc2, dec2, PIV_PAD_IDX, device).to(device)

models = [model1, model2]
fields = [EN_FIELD, ES_FIELD, FR_FIELD]
model = PivotSeq2Seq(models, fields, device).to(device)

#### Triangulate

In [ ]:
EMB_DIM = 256
HID_DIM = 512
DROPOUT = 0.5

# DIRECT
INPUT_DIM = len(EN_FIELD.vocab)
OUTPUT_DIM = len(FR_FIELD.vocab)
SRC_PAD_IDX = EN_FIELD.vocab.stoi[EN_FIELD.pad_token]

attn = Attention(HID_DIM, HID_DIM)
enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM, HID_DIM, DROPOUT)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, HID_DIM, DROPOUT, attn)

model_direct = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)
model_direct.load_state_dict(torch.load(f'{DIR_PATH}/seq2seq-EnFr-1.pt')['model_state_dict'])

# PIVOT
INPUT_DIM = len(EN_FIELD.vocab)
PIV_DIM = len(FIELD_DICT['it'].vocab)
OUTPUT_DIM = len(FR_FIELD.vocab)

SRC_PAD_IDX = EN_FIELD.vocab.stoi[EN_FIELD.pad_token]
attn1 = Attention(HID_DIM, HID_DIM)
enc1 = Encoder(INPUT_DIM, EMB_DIM, HID_DIM, HID_DIM, DROPOUT)
dec1 = Decoder(PIV_DIM, EMB_DIM, HID_DIM, HID_DIM, DROPOUT, attn1)
model1 = Seq2Seq(enc1, dec1, SRC_PAD_IDX, device).to(device)

PIV_PAD_IDX = FIELD_DICT['it'].vocab.stoi[FIELD_DICT['it'].pad_token]
attn2 = Attention(HID_DIM, HID_DIM)
enc2 = Encoder(PIV_DIM, EMB_DIM, HID_DIM, HID_DIM, DROPOUT)
dec2 = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, HID_DIM, DROPOUT, attn2)
model2 = Seq2Seq(enc2, dec2, PIV_PAD_IDX, device).to(device)

models = [model1, model2]
fields = [EN_FIELD, FIELD_DICT['it'], FR_FIELD]
model_piv = PivotSeq2Seq(models, fields, device).to(device)
model_piv.load_state_dict(torch.load(f'{DIR_PATH}/piv-EnItFr.pt')['model_state_dict'])

# TRIANGULATE
models = [model_direct, model_piv]
model = TriangSeq2Seq(models, OUTPUT_DIM, device, alpha=1.1, method='weighted_1', train_backbone=True).to(device)

### Train loop

In [ ]:
# if isinstance(model, Seq2Seq) or isinstance(model, PivotSeq2Seq) or (isinstance(model, TriangSeq2Seq) and model.train_backbone):
#   model.apply(init_weights) # remove if backbone models are freezed
#   print('init_weights success')
# elif isinstance(model, TriangSeq2Seq) and not model.train_backbone:
#   model.head.apply(init_weights)
#   print('init_weights head ONLY success')
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 87,319,499 trainable parameters


In [ ]:
criterion2 = nn.CrossEntropyLoss(ignore_index = FR_FIELD.vocab.stoi[FR_FIELD.pad_token])

if isinstance(model, PivotSeq2Seq) or isinstance(model, TriangSeq2Seq):
  criterion1 = nn.CrossEntropyLoss(ignore_index = FIELD_DICT['it'].vocab.stoi[FIELD_DICT['it'].pad_token])
  criterion3 = [criterion1, criterion2]
  print('Created criterion for piv')

if isinstance(model, TriangSeq2Seq):
  criterion4 = {
      'model_0': criterion2,
      'model_1': criterion3,
      'TRG': nn.CrossEntropyLoss(ignore_index = FR_FIELD.vocab.stoi[FR_FIELD.pad_token])
  }
  print('Created criterion for triang')

Created criterion for piv
Created criterion for triang


In [ ]:
N_EPOCHS = 5
CLIP = 1
best_valid_loss = float('inf')
best_train_loss = float('inf')
model_name = 'triang-EnFr-EnItFr-w1-trainall.pt'
train_log = []

In [ ]:
LR = 0.001
optimizer = optim.Adam(model.parameters(), lr=LR)
# scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2, 5, 7, 8, 9], gamma=2.0/3.0) PIV TRAINING w/ LR = 0.0012
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1, 2, 4], gamma=1.0/3.0)

In [ ]:
# model_path = f'{DIR_PATH}/{model_name}'
# ckpt = torch.load(model_path)
# model.load_state_dict(ckpt['model_state_dict'])

<All keys matched successfully>

In [ ]:
for epoch in range(N_EPOCHS):
  if isinstance(model, Seq2Seq):
    train_loss = trainSeq2Seq(model, train_iterator, optimizer, criterion2, CLIP)
    valid_loss = evaluateSeq2Seq(model, valid_iterator, criterion2)
  elif isinstance(model, PivotSeq2Seq):
    train_loss = trainPivot(model, train_iterator, optimizer, criterion3, CLIP)
    valid_loss = evaluatePivot(model, valid_iterator, criterion3)
  elif isinstance(model, TriangSeq2Seq):
    train_loss = trainTriang(model, train_iterator, optimizer, criterion4, CLIP)
    valid_loss = evaluateTriang(model, valid_iterator, criterion4)
  else: raise Exception('Model type is unknown')

  epoch_info = [model_name, dataname, scheduler.get_last_lr()[0], BATCH_SIZE, HID_DIM, DROPOUT, epoch, N_EPOCHS, train_loss, valid_loss]
  train_log.append([str(info) for info in epoch_info])
  train_log = update_trainlog(train_log, f'{DIR_PATH}/training_log-OLD.txt')

  scheduler.step()

  # if train_loss < best_train_loss or valid_loss < best_valid_loss:
  if valid_loss < best_valid_loss:
    best_train_loss = train_loss
    best_valid_loss = valid_loss
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()
    }, f'{DIR_PATH}/{model_name}')
    print('SAVED MODEL')

  print(f'Epoch: {epoch:02} \t Train Loss: {train_loss:.3f} \t Val. Loss: {valid_loss:.3f}')

100%|██████████| 200/200 [00:29<00:00,  6.76it/s]


update_trainlog SUCCESS
SAVED MODEL
Epoch: 00 	 Train Loss: 20.115 	 Val. Loss: 29.692


100%|██████████| 200/200 [00:29<00:00,  6.81it/s]


update_trainlog SUCCESS
SAVED MODEL
Epoch: 01 	 Train Loss: 11.040 	 Val. Loss: 21.761


100%|██████████| 200/200 [00:29<00:00,  6.81it/s]


update_trainlog SUCCESS
SAVED MODEL
Epoch: 02 	 Train Loss: 7.937 	 Val. Loss: 19.412


100%|██████████| 200/200 [00:29<00:00,  6.83it/s]


update_trainlog SUCCESS
Epoch: 03 	 Train Loss: 7.719 	 Val. Loss: 19.493


100%|██████████| 200/200 [00:29<00:00,  6.76it/s]


update_trainlog SUCCESS
SAVED MODEL
Epoch: 04 	 Train Loss: 6.722 	 Val. Loss: 18.753


In [ ]:
LR = scheduler.get_last_lr()[0]
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1], gamma=1.0/3.0)
scheduler.get_last_lr()

[3.703703703703703e-05]

In [ ]:
for epoch in range(2):
  if isinstance(model, Seq2Seq):
    train_loss = trainSeq2Seq(model, train_iterator, optimizer, criterion2, CLIP)
    valid_loss = evaluateSeq2Seq(model, valid_iterator, criterion2)
  elif isinstance(model, PivotSeq2Seq):
    train_loss = trainPivot(model, train_iterator, optimizer, criterion3, CLIP)
    valid_loss = evaluatePivot(model, valid_iterator, criterion3)
  elif isinstance(model, TriangSeq2Seq):
    train_loss = trainTriang(model, train_iterator, optimizer, criterion4, CLIP)
    valid_loss = evaluateTriang(model, valid_iterator, criterion4)
  else: raise Exception('Model type is unknown')

  epoch_info = [model_name, dataname, scheduler.get_last_lr()[0], BATCH_SIZE, HID_DIM, DROPOUT, epoch, N_EPOCHS, train_loss, valid_loss]
  train_log.append([str(info) for info in epoch_info])
  train_log = update_trainlog(train_log, f'{DIR_PATH}/training_log-OLD.txt')

  scheduler.step()

  # if train_loss < best_train_loss or valid_loss < best_valid_loss:
  if valid_loss < best_valid_loss:
    best_train_loss = train_loss
    best_valid_loss = valid_loss
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()
    }, f'{DIR_PATH}/{model_name}')
    print('SAVED MODEL')

  print(f'Epoch: {epoch:02} \t Train Loss: {train_loss:.3f} \t Val. Loss: {valid_loss:.3f}')

 22%|██▏       | 882/4000 [04:02<18:23,  2.83it/s]

## Eval

In [ ]:
for batch in test_iterator:
  break
batch.fields

dict_keys(['en', 'fr', 'it'])

In [ ]:
model_path = f'{DIR_PATH}/piv-EnEsFr.pt'
ckpt = torch.load(model_path)

In [ ]:
# optimizer.load_state_dict(ckpt['optimizer_state_dict'])
# scheduler.load_state_dict(ckpt['scheduler_state_dict'])
model.load_state_dict(ckpt['model_state_dict']) # strict=False if some dimensions are different

<All keys matched successfully>

In [ ]:
valid_loss = evaluateTriang(model, valid_iterator, criterions)
print(f'Test Loss: {valid_loss:.3f}')

## Inference

In [ ]:
def sent2tensor(src_field, trg_field, device, max_len, sentence=None):
  if sentence != None:
    if isinstance(sentence, str):
      tokens = tokenize_en(sentence)
    else:
      tokens = [token.lower() for token in sentence]
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)  # [seq_len, N] w/ N=1 for batch
    src_len_tensor = torch.LongTensor([len(src_indexes)]).to(device)
    return src_tensor, src_len_tensor

  trg_tensor = torch.LongTensor([trg_field.vocab.stoi[trg_field.init_token]] + [0 for i in range(1, max_len)]).view(-1, 1).to(device) # [seq_len, 1]
  trg_len_tensor = torch.LongTensor([max_len]).to(device)
  return (trg_tensor, trg_len_tensor)

def idx2sent(trg_field, arr):
  n_sents = arr.shape[1]  # arr = [seq_len, N]
  results = []
  for i in range(n_sents):  # for each sent
    pred_sent = []
    pred = arr[:, i]
    for i in pred[1:]:  # for each word
      pred_sent.append(trg_field.vocab.itos[i])
      if i == trg_field.vocab.stoi[trg_field.eos_token]: break
    results.append(pred_sent)
  return results

### Translate by sent

In [ ]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len):
  model.eval()
  with torch.no_grad():
    # create data
    src = sent2tensor(src_field, trg_field, device, max_len, sentence)
    trg = sent2tensor(src_field, trg_field, device, max_len)
    # get data
    if isinstance(model, Seq2Seq):
      data = [src, trg]
    elif isinstance(model, PivotSeq2Seq):
      data = [src] + [trg for _ in range(model.num_model)]
    elif isinstance(model, TriangSeq2Seq):
      data = {'TRG': trg}
      for i in range(model.num_model):
        submodel = getattr(model, f'model_{i}')
        if isinstance(submodel, Seq2Seq):
          data[f'model_{i}'] = [src, trg]
        elif isinstance(submodel, PivotSeq2Seq):
          data[f'model_{i}'] = [src] + [trg for _ in range(submodel.num_model)]
        else: raise Exception('Only support Seq2Seq & PivotSeq2Seq nested inside TriangSeq2Seq')
    else: raise Exception('Model type is unknown')
    # feed model
    output = model(data, None, 0) # output = [trg_len, N, dec_emb_dim] w/ N=1
    output = output.argmax(-1).detach().cpu().numpy() # output = [seq_len, N]
    results = idx2sent(trg_field, output)
    return results

In [ ]:
def translate_sentence_seq2seq(sentence, src_field, trg_field, model: Seq2Seq, device, max_len=64):
  model.eval()
  with torch.no_grad():
    # get data
    src_tensor, src_len_tensor = sent2tensor(src_field, trg_field, device, max_len, sentence)
    trg_tensor, trg_len_tensor = sent2tensor(src_field, trg_field, device, max_len)
    data = [(src_tensor, src_len_tensor), (trg_tensor, trg_len_tensor)]
    # feed model
    output = model(data, None, 0) # output = [trg_len, N, dec_emb_dim] w/ N=1
    output = output.argmax(-1).detach().cpu().numpy() # output = [seq_len, N]
    results = idx2sent(trg_field, output)
    return results

def translate_sentence_pivot(sentence, src_field, trg_field, model, device, max_len=64):  # not yet modified
  model.eval()
  with torch.no_grad():
    # get data
    src_tensor, src_len_tensor = sent2tensor(src_field, trg_field, device, max_len, sentence)
    trg_tensor, trg_len_tensor = sent2tensor(src_field, trg_field, device, max_len)
    data = [(src_tensor, src_len_tensor)] + [(trg_tensor.clone().detach().to(device), trg_len_tensor.clone().detach().to(device)) for _ in range(model.num_model)]
    # feed model
    output = model(data, None, 0) # output = [trg_len, N, dec_emb_dim]
    output = output.argmax(-1).detach().cpu().numpy()
    results = idx2sent(trg_field, output)
    return results

def translate_batch_triang(sentence, src_field, trg_field, model, device, max_len=64):  # not yet complete. modify sentence/sentences, fields base on type of models
  model.eval()
  with torch.no_grad():
    # get data
    src_tensor, src_len_tensor = sent2tensor(src_field, trg_field, device, max_len, sentence)
    trg_tensor, trg_len_tensor = sent2tensor(src_field, trg_field, device, max_len)
    data = {
      'model_0': [(src_tensor, src_len_tensor), (trg_tensor, trg_len_tensor)],
      'model_1': [(src_tensor, src_len_tensor), (trg_tensor, trg_len_tensor), (trg_tensor, trg_len_tensor)],
      'TRG': (trg_tensor, trg_len_tensor)}
    # feed model
    output = model(data, None, 0)
    output = output.argmax(-1).detach().cpu().numpy()
    results = idx2sent(trg_field, output)
    return results

In [ ]:
if isinstance(model, Seq2Seq):
  translator = translate_sentence_seq2seq
  print('Created translator for seq2seq')
elif isinstance(model, PivotSeq2Seq):
  translator = translate_sentence_pivot
  print('Created translator for piv')
elif isinstance(model, TriangSeq2Seq):
  translator = translate_batch_triang
  print('Created translator for triang')
else: raise Exception('Model type is unknown')

Created translator for triang


In [ ]:
example_idx = 2132
src = vars(valid_dt.examples[example_idx])['en']
trg = vars(valid_dt.examples[example_idx])['fr']
pred = translate_sentence(src, EN_FIELD, FR_FIELD, model, device, 50)
print(src)
print(trg)
print(pred)

['(', 'da', ')', 'mr', 'president', ',', 'i', 'merely', 'wish', 'to', 'add', 'to', 'what', 'has', 'already', 'been', 'said', '.']
['-', '(', 'da', ')', 'monsieur', 'le', 'président', ',', 'je', 'souhaite', 'simplement', 'ajouter', 'quelques', 'mots', 'à', 'ce', 'qui', 'a', 'déjà', 'été', 'dit', '.']
[['(', 'da', ')', 'monsieur', 'le', 'président', ',', 'je', 'voudrais', 'juste', 'ajouter', 'quelques', 'ce', 'qui', 'a', 'déjà', 'été', 'dit', '.', '<eos>']]


### Translate by batch

In [ ]:
test_iterator_1 = BucketIterator(
     test_dt,
     batch_size = 4,
     sort_within_batch = True,
     sort_key = lambda x : len(x.fr),
     device = device)

for batch in test_iterator_1: break

print(batch.fields)
print(batch.en[1])
print(batch.fr[1])

dict_keys(['en', 'fr'])
tensor([21, 20, 23, 15], device='cuda:0')
tensor([24, 24, 24, 24], device='cuda:0')


In [ ]:
def translate_batch_seq2seq(model_infer, iterator, trg_field, device):
  model_infer.eval()
  with torch.no_grad():
    gt_sents = []
    pred_sents = []
    for i, batch in enumerate(tqdm(iterator)):
      data = [batch.en, batch.fr] # modify based on model
      output = model_infer(data, criterion=None, teacher_forcing_ratio=0)

      pred = output.argmax(-1).detach().cpu().numpy() # [seq_len, N]
      truth = batch.fr[0].detach().cpu().numpy()  # [seq_len, N]

      gt_sents = gt_sents + idx2sent(trg_field, truth)
      pred_sents = pred_sents + idx2sent(trg_field, pred)
      if i==10: break
    return gt_sents, pred_sents

In [ ]:
gt_sents, pred_sents = translate_batch_seq2seq(model_infer, test_iterator_1, FR_FIELD, device)

  1%|          | 10/1600 [00:00<00:31, 49.92it/s]


In [ ]:
for i, (gt_sent, pred_sent) in enumerate(zip(gt_sents, pred_sents)):
  print(gt_sent)
  print(pred_sent)
  print()
  if i==10: break

['1', '.', '<eos>']
['1', '.', '<eos>']

['1', '.', '<eos>']
['1', '.', '<eos>']

['certes', '.', '<eos>']
['cela', 'ne', 'fait']

['2', '.', '<eos>']
['2', '.', '<eos>']

["c'", 'est', 'un', 'outil', 'important', 'pour', 'rendre', 'compte', 'aux', 'citoyens', 'de', "l'", 'europe', ',', 'à', 'leurs', 'représentants', 'élus', ',', 'au', 'parlement', ',', 'de', 'la', 'politique', 'monétaire', 'et', 'de', 'nos', 'activités', 'dans', 'les', 'domaines', 'de', 'notre', 'compétence', '.', '<eos>']
['il', 's’', 'agit', 'là', 'un', 'instrument', 'important', 'pour', 'tenir', 'un', 'signe', 'de', 'citoyens', 'européens', ',', 'en', 'leurs', 'représentants', 'élus', ',', 'au', 'parlement', ',', 'de', 'la', 'politique', 'de', 'nos', 'politiques', 'en', 'matière', 'de', 'responsabilité', '.', '<eos>']

['à', 'cet', 'égard', ',', 'le', 'fait', 'de', 'soutenir', "l'", 'appel', 'de', 'la', 'commission', 'pour', 'introduire', 'des', 'niveaux', 'de', 'référence', 'communs', 'pour', 'la', 'sécurité', 'nu

## BLEU

### Main

In [ ]:
def calculate_bleu(translator, data, src_field, trg_field, model, device, max_len=50):
  trgs = []
  pred_trgs = []
  for i, datum in enumerate(tqdm(data)):
    src = vars(datum)['en']
    trg = vars(datum)['fr']
    pred = translator(src, src_field, trg_field, model, device, max_len)
    pred_trgs.append(pred[0][:-1])  #cut off <eos> token
    trgs.append([trg])
    # if i==2000: break
  return pred_trgs, trgs

In [ ]:
pred_trgs, trgs = calculate_bleu(translate_sentence, test_dt, EN_FIELD, FR_FIELD, model, device)

100%|██████████| 6400/6400 [16:06<00:00,  6.62it/s]


In [ ]:
score = bleu_score(pred_trgs, trgs)
print(f'BLEU score = {score*100:.2f}')

BLEU score = 26.12


### IF error happens: index 4 is out of bounds for dimension 0 with size 4 ==> use this

In [ ]:
def get_pred_trg_corpus(data, model_infer, device, SRC_FIELD, TRG_FIELD, PIV_FIELD=None, max_len=50):
  candidate_corpus = []
  references_corpus = []
  for datum in tqdm(data):
    src = vars(datum)['src']
    trg = vars(datum)['trg']
    if isinstance(model_infer, PivotSeq2Seq):
      piv = vars(datum)['piv']
      sent1, sent = translate_sentence_1piv(src, SRC_FIELD, PIV_FIELD, TRG_FIELD, model_infer, device, max_len=max_len)
    else:
      sent, attn = translate_sentence(src, SRC_FIELD, TRG_FIELD, model_infer, device, max_len=max_len)
    candidate_corpus.append(sent[:-1])
    references_corpus.append([trg])
  return candidate_corpus, references_corpus

In [ ]:
candidate_corpus, references_corpus = get_pred_trg_corpus(test_dt, model_infer, device, SRC_FIELD, TRG_FIELD, PIV_FIELD)

100%|██████████| 6400/6400 [02:43<00:00, 39.08it/s]


In [ ]:
candidate_corpus[:1376], references_corpus[:1376]

In [ ]:
i = 5000
j = 100
k = 30000
bleu_score(candidate_corpus[: i]+candidate_corpus[i+j:k], references_corpus[: i]+references_corpus[i+j:k])

0.26010842469504236

In [ ]:
candidate = ['hello', '']
references = [['hello'], ['.']]
bleu_score(candidate, references)

0.8187307530779819

### Result

1. Seq2Seq
  * seq2seq-EnFr-1.pt: BLEU = 38.74
1. PivotSeq2Seq
  * piv_EnItFr.pt: BLEU = 31.37
  * piv-EnEsFr.pt: BLEU = 32.27
  * piv-EnRoFr.pt: BLEU = 29.51
  * piv-EnPtFr.pt: BLEU = 31.36
1. TriangSeq2Seq
  * triang-EnFr-EnItFr:
    * triang-EnFr-EnItFr-1.pt: BLEU = 26.12
    * triang-EnFr-EnItFr.pt: BLEU = 31.95 (EnFr: BLEU=38.03, EnItFr: BLEU=24.62)
  * triang-EnFr-EnEsFr: (Prof Neller's suggestion, method='max')
    * BLEU = 37.80 (prod)
    * BLEU = 37.91 (mean)
  * triang-EnFr-EnPtFr: (Prof Neller's suggestion, method='max')
    * BLEU = 37.81 (prod)
    * BLEU = 37.93 (mean)
  * triang-EnFr-EnRoFr: (Prof Neller's suggestion, method='max')
    * BLEU = 37.98 (prod)
    * BLEU = 38.09 (mean)
  * triang-EnFr-EnItFr-EnEsFr: (Prof Neller's suggestion, method='max')
    * BLEU = 36.25 (mean)

* attn_en-fr_32k.pt: BLEU = 12.65
* attn_enfr_160kset.pt: BLEU = 32.18
* piv_endefr_74kset_2.pt: BLEU = 26.33


# End